In [1]:
import pandas as pd
import numpy as np

In [2]:
cs = pd.read_csv('data/cs-training_ver02.csv')
cs.head()

,seriousdlqin2yrs,revolvingutilizationofunsecuredlines,age,numberoftime30-59dayspastduenotworse,debtratio,monthlyincome,numberofopencreditlinesandloans,numberoftimes90dayslate,numberrealestateloansorlines,numberoftime60-89dayspastduenotworse,numberofdependents
0,1.0,0.766127,45,2.0,0.802982,9120.0,13,0.0,6,0.0,2.0
1,0.0,0.957151,40,0.0,0.121876,2600.0,4,0.0,0,0.0,1.0
2,0.0,0.658180,38,1.0,0.085113,3042.0,2,1.0,0,0.0,0.0
3,0.0,0.233810,30,0.0,0.036050,3300.0,5,0.0,0,0.0,0.0
4,0.0,0.907239,49,1.0,0.024926,63588.0,7,0.0,1,0.0,0.0


In [3]:
cs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 11 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   seriousdlqin2yrs                      150000 non-null  float64
 1   revolvingutilizationofunsecuredlines  150000 non-null  float64
 2   age                                   150000 non-null  int64  
 3   numberoftime30-59dayspastduenotworse  150000 non-null  float64
 4   debtratio                             150000 non-null  float64
 5   monthlyincome                         150000 non-null  float64
 6   numberofopencreditlinesandloans       150000 non-null  int64  
 7   numberoftimes90dayslate               150000 non-null  float64
 8   numberrealestateloansorlines          150000 non-null  int64  
 9   numberoftime60-89dayspastduenotworse  150000 non-null  float64
 10  numberofdependents                    150000 non-null  float64
dtype

In [4]:
y = cs.seriousdlqin2yrs
x = cs.drop(columns=['seriousdlqin2yrs'])

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [7]:
pipe = Pipeline([('전처리',StandardScaler()),('알고리즘', SVC())])

In [13]:
c_range = np.concatenate([np.arange(1,11)/100,np.arange(2,11)/10,np.arange(2,11)])

In [16]:
param = [
    {'알고리즘': [LogisticRegression()],
     '전처리' : [StandardScaler(), MinMaxScaler(), None],
     '알고리즘__C' : c_range,
     '알고리즘__max_iter' : [300],
     '알고리즘__solver' : ['lbfgs','liblinear','saga']},
    {'알고리즘': [KNeighborsClassifier()],
     '전처리': [StandardScaler(), MinMaxScaler(), None],
     '알고리즘__n_neighbors' : np.arange(1,100),
     '알고리즘__weights' : ['uniform','distance'],
     '알고리즘__p' : [1, 2]},
    {'알고리즘':[SVC()],
     '전처리': [StandardScaler(), MinMaxScaler(), None],
     '알고리즘__C' : c_range},
    {'알고리즘':[RandomForestClassifier()],
     '전처리': [MinMaxScaler(), None],
     '알고리즘__n_estimators':[300],
     '알고리즘__criterion':['gini','entropy'],
     '알고리즘__max_depth':[None]+list(np.arange(5,100))}]

In [18]:
np.random.seed(123)
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=pipe, param_grid=param,
                    scoring='roc_auc', cv=4,
                    n_jobs=-1, verbose=1)
grid.fit(x,y)

Fitting 4 folds for each of 1908 candidates, totalling 7632 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    4.5s


KeyboardInterrupt: 

In [20]:
from sklearn.model_selection import RandomizedSearchCV
grid = RandomizedSearchCV(estimator=pipe, param_distributions=param,
                          scoring='roc_auc', cv=4,
                          n_iter=25, n_jobs=-1, verbose=1)
grid.fit(x,y)

Fitting 4 folds for each of 25 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  6.3min finished


RandomizedSearchCV(cv=4,
                   estimator=Pipeline(steps=[('전처리', StandardScaler()),
                                             ('알고리즘', SVC())]),
                   n_iter=25, n_jobs=-1,
                   param_distributions=[{'알고리즘': [LogisticRegression(C=8.0,
                                                                     max_iter=300)],
                                         '알고리즘__C': array([ 0.01,  0.02,  0.03,  0.04,  0.05,  0.06,  0.07,  0.08,  0.09,
        0.1 ,  0.2 ,  0.3 ,  0.4 ,  0.5 ,  0.6 ,  0.7 ,  0.8 ,  0.9 ,
        1.  ,  2.  ,  3.  ,  4.  ,  5.  ,  6.  ,  7.  ,  8.  ,  9.  ,
       10.  ]),
                                         '알고리즘__max_iter': [300],
                                         '알고리즘__solver'...
        0.1 ,  0.2 ,  0.3 ,  0.4 ,  0.5 ,  0.6 ,  0.7 ,  0.8 ,  0.9 ,
        1.  ,  2.  ,  3.  ,  4.  ,  5.  ,  6.  ,  7.  ,  8.  ,  9.  ,
       10.  ]),
                                         '전처리': [StandardScaler(),
           

In [22]:
cv_result = pd.DataFrame(grid.cv_results_)
cv_result

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_전처리,param_알고리즘__weights,param_알고리즘__p,param_알고리즘__n_neighbors,param_알고리즘,param_알고리즘__n_estimators,...,param_알고리즘__solver,param_알고리즘__max_iter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,20.806225,2.634752,117.130218,4.887505,StandardScaler(),uniform,2,65,KNeighborsClassifier(),NaN,...,NaN,NaN,"{'전처리': StandardScaler(), '알고리즘__weights': 'un...",0.843806,0.841288,0.843033,0.846596,0.843681,0.001914,6
1,5.122254,2.219900,45.903873,2.001002,MinMaxScaler(),uniform,1,67,KNeighborsClassifier(),NaN,...,NaN,NaN,"{'전처리': MinMaxScaler(), '알고리즘__weights': 'unif...",0.823840,0.829853,0.829631,0.830255,0.828395,0.002639,12
2,2.690250,0.235846,3.838500,0.128956,None,distance,2,51,KNeighborsClassifier(),NaN,...,NaN,NaN,"{'전처리': None, '알고리즘__weights': 'distance', '알고...",0.647169,0.646281,0.639101,0.645588,0.644535,0.003187,22
3,4.983503,1.966863,38.702623,1.391047,MinMaxScaler(),distance,1,42,KNeighborsClassifier(),NaN,...,NaN,NaN,"{'전처리': MinMaxScaler(), '알고리즘__weights': 'dist...",0.810477,0.816637,0.814150,0.816965,0.814557,0.002595,16
4,95.711147,1.896784,2.741378,0.070842,MinMaxScaler(),NaN,NaN,NaN,RandomForestClassifier(),300,...,NaN,NaN,"{'전처리': MinMaxScaler(), '알고리즘__n_estimators': ...",0.842453,0.841424,0.845111,0.847996,0.844246,0.002549,5
5,16.030499,3.058440,169.652741,4.631505,StandardScaler(),uniform,1,95,KNeighborsClassifier(),NaN,...,NaN,NaN,"{'전처리': StandardScaler(), '알고리즘__weights': 'un...",0.847236,0.845383,0.847966,0.849948,0.847633,0.001635,2
6,114.050987,1.683450,16.594753,0.053700,None,NaN,NaN,NaN,SVC(),NaN,...,NaN,NaN,"{'전처리': None, '알고리즘__C': 4.0, '알고리즘': SVC()}",0.488859,0.604133,0.621734,0.567584,0.570577,0.051063,25
7,4.046174,1.599321,21.668938,0.726159,MinMaxScaler(),uniform,1,5,KNeighborsClassifier(),NaN,...,NaN,NaN,"{'전처리': MinMaxScaler(), '알고리즘__weights': 'unif...",0.711707,0.723558,0.720442,0.723304,0.719753,0.004804,18
8,1.736500,0.025849,2.822250,0.094270,None,uniform,1,18,KNeighborsClassifier(),NaN,...,NaN,NaN,"{'전처리': None, '알고리즘__weights': 'uniform', '알고리...",0.636753,0.642115,0.641631,0.645980,0.641620,0.003277,23
9,1.836499,0.032331,3.893500,0.108423,None,distance,2,71,KNeighborsClassifier(),NaN,...,NaN,NaN,"{'전처리': None, '알고리즘__weights': 'distance', '알고...",0.654366,0.650969,0.643232,0.648542,0.649277,0.004057,21


In [24]:
cv_result.sort_values('rank_test_score').iloc[0]

mean_fit_time                                                           1.487
std_fit_time                                                         0.126457
mean_score_time                                                       0.02775
std_score_time                                                     0.00148046
param_전처리                                                      MinMaxScaler()
param_알고리즘__weights                                                       NaN
param_알고리즘__p                                                             NaN
param_알고리즘__n_neighbors                                                   NaN
param_알고리즘                            LogisticRegression(C=8.0, max_iter=300)
param_알고리즘__n_estimators                                                  NaN
param_알고리즘__max_depth                                                     NaN
param_알고리즘__criterion                                                     NaN
param_알고리즘__C                                                   

In [25]:
grid.best_score_

0.8487193111218351

In [26]:
grid.best_params_

{'전처리': MinMaxScaler(),
 '알고리즘__solver': 'lbfgs',
 '알고리즘__max_iter': 300,
 '알고리즘__C': 8.0,
 '알고리즘': LogisticRegression(C=8.0, max_iter=300)}

In [29]:
best_grid = grid.best_estimator_
best_grid

Pipeline(steps=[('전처리', MinMaxScaler()),
                ('알고리즘', LogisticRegression(C=8.0, max_iter=300))])

In [31]:
import joblib
with open('baseline.pkl', 'wb') as f:
    joblib.dump({'grid_result':grid, 'best_param':grid.best_params_,
                 'cv_result':cv_result, 'best_baseline':best_grid}, f)